## Agents

### 🤖 What is an Agent?

- An LLM configured with specific instructions to do a task. It may use tools to achieve more specific goals.

#### install

In [ ]:
# ! pip install openai-agents nest_asyncio


### imports

In [ ]:
from pydantic import BaseModel
import dotenv
import nest_asyncio
from agents import Agent, Runner, GuardrailFunctionOutput, function_tool
from openai.types.responses import ResponseTextDeltaEvent
import asyncio
from typing import AsyncGenerator
import requests
import json
from IPython.display import display, Markdown, DisplayHandle

##### Apply nest_asyncio to allow asyncio event loops to be nested

In [ ]:
nest_asyncio.apply()

#### Load environment variables

In [ ]:
dotenv.load_dotenv()

##### function tool

In [ ]:
@function_tool
def get_usd_base_rate() -> dict:
  """
  Get today's USD base rate.
  """
  response: dict = requests.get("https://api.vatcomply.com/rates?base=USD")
  if response.status_code != 200:
    return f"Error: {response.status_code}"
  return response.json()

### Agent with tool

In [ ]:
currency_agent = Agent(
  name="Currency Agent",
  model='gpt-4o-mini',
  tools=[get_usd_base_rate],
  instructions="""You are an expert financial assistant, with expertise in currency conversion. To get the latest USD base rate, use the get_usd_base_rate function.
Provide the user with answers in their requested currency and USD equivalent.

Example: 
User: How much does it cost to buy a tesla in London?
Assistant: The cost of a Tesla in London is £50,000. This is equivalent to $65,000 USD based on the current exchange rate.
"""
)

### Ask the agent

In [12]:
async def ask_ai(q: str) -> AsyncGenerator[str, None]:      
    result = Runner.run_streamed(currency_agent, input=q)
    async for event in result.stream_events():
        if event.type == "raw_response_event" and isinstance(event.data, ResponseTextDeltaEvent):
            yield event.data.delta

##### question

In [13]:
question: str = "How much does it cost to buy a tesla in Canada?"

#### Streaming Response (with Markdown)

In [14]:
full_response: str = ""
display_id: DisplayHandle = display(Markdown(full_response), display_id=True)

async for chunk in ask_ai(question):
    full_response += chunk
    display_id.update(Markdown(full_response))
    await asyncio.sleep(0.01)

full_response

The cost of a Tesla in Canada is typically around CAD 70,000. Based on the current exchange rate of 1 CAD = 0.723 USD, this is equivalent to approximately $51,090 USD.

'The cost of a Tesla in Canada is typically around CAD 70,000. Based on the current exchange rate of 1 CAD = 0.723 USD, this is equivalent to approximately $51,090 USD.'